# Porównanie implementacji Longstaff–Schwartz z wartościami z artykułu (Tabela 1)

Ten notebook liczy cenę amerykańskiego **puta** metodą Longstaff–Schwartz
dla parametrów podanych w Tabeli 1 pracy *Longstaff & Schwartz (2001)*
i porównuje je z wartościami referencyjnymi z tej tabeli.


## 1  – Importy i moduł `lsm`
Załaduj najpierw swój moduł `lsm.py` (z funkcją `lsm_price_multi` i fabryką `payoff_put`).

In [9]:

import math
import pandas as pd
import math, pandas as pd, sys, pathlib
import models.longstaff_schwartz as lsm

# upewnij się, że lsm.py jest w tym samym katalogu
import importlib, sys, pathlib
sys.path.append(str(pathlib.Path.cwd()))
import numpy as np
np.random.seed(16)

#print("lsm module loaded, version:", getattr(lsm, "__version__", "n/a"))


SyntaxError: invalid syntax (2055649913.py, line 4)

## 2  – Dane referencyjne (Tabela 1)

In [10]:

# (S0, sigma, T, american_value) z Longstaff & Schwartz (2001), Table 1
reference = [
    (36 , 0.20 , 1 , 4.478),
    (36 , 0.20 , 2 , 4.840),
    (36 , 0.40 , 1 , 7.101),
    (36 , 0.40 , 2 , 8.508),
    (38 , 0.20 , 1 , 3.250),
    (38 , 0.20 , 2 , 3.745),
    (38 , 0.40 , 1 , 6.148),
    (38 , 0.40 , 2 , 7.670),
    (40 , 0.20 , 1 , 2.314),
    (40 , 0.20 , 2 , 2.885),
    (40 , 0.40 , 1 , 5.312),
    (40 , 0.40 , 2 , 6.920),
]


## 3  – Parametry wspólne dla symulacji

In [11]:

RISK_FREE = 0.06
STRIKE    = 40
M_STEPS   = 50 # 100 exercise points per year
N_PATHS   = 100000
SEED      = 123


## 4  – Obliczenia LS‑MC i zestawienie wyników

In [ ]:
import sys
sys.path.append('../../..')
from amopt.models import longstaff_schwartz as lsm
rows = []
for S0, sigma, T, ref_val in reference:
    model_val = lsm.lsm_price_multi(
        S0=[S0], r=RISK_FREE, sigma=[sigma], corr=[[1]], T=T,
        M=int(M_STEPS * T),
        N=N_PATHS,
        payoff_fn=lsm.payoff_put(STRIKE),
        seed=SEED
    )
    abs_err = model_val - ref_val
    rel_err = abs(abs_err) / ref_val * 100
    rows.append(dict(
        S0=S0, sigma=sigma, T=T,
        Paper=round(ref_val, 3),
        Ours=round(model_val, 3),
        AbsErr=round(abs_err, 3),
        RelErrPct=round(rel_err, 2)
    ))

df = pd.DataFrame(rows)
df


In [5]:
lsm.lsm_price_multi(
        S0=[120], r=0.08, sigma=0.2, corr=[[1]], T=0.5,
        M=int(M_STEPS * 0.5),
        N=N_PATHS,
        payoff_fn=lsm.payoff_call(100),
        seed=SEED
    )

np.float64(24.301027935969227)

### Interaktywna tabela (tylko w ChatGPT / Edu‑Jupyter)

## 5  – Zapis do pliku CSV

In [6]:

csv_path = "lsm_vs_paper2.csv"
df.to_csv(csv_path, index=False)
print("Zapisano:", csv_path)


Zapisano: lsm_vs_paper2.csv


In [7]:

# --- Black–Scholes European put --------------------------
def bs_eur_put(S, K, r, sigma, T):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    N = lambda x: 0.5 * (1 + math.erf(x / math.sqrt(2)))
    call = S * N(d1) - K * math.exp(-r * T) * N(d2)
    return call + K * math.exp(-r * T) - S


In [8]:


rows = []
for S0, sigma, T, ref_val in reference:
    american = lsm.lsm_price_multi(
        S0=[S0], r=RISK_FREE, sigma=[sigma], corr=[[1]], T=T,
        M=int(M_STEPS * T), N=N_PATHS,
        payoff_fn=lsm.payoff_put(STRIKE), seed=SEED
    )
    european = bs_eur_put(S0, STRIKE, RISK_FREE, sigma, T)
    rows.append(dict(
        S0=S0,
        sigma=sigma,
        T=T,
        Paper_AM=ref_val,
        Our_AM=round(american, 3),
        Eur_Put=round(european, 3),
        AM_minus_EUR=round(american - european, 3),
        RelErr_vs_Paper_pct=round(abs(american - ref_val)/ref_val*100, 2)
    ))

df = pd.DataFrame(rows)
df


,S0,sigma,T,Paper_AM,Our_AM,Eur_Put,AM_minus_EUR,RelErr_vs_Paper_pct
0,36,0.2,1,4.478,4.453,3.844,0.609,0.56
1,36,0.2,2,4.840,4.812,3.763,1.049,0.59
2,36,0.4,1,7.101,7.083,6.711,0.372,0.25
3,36,0.4,2,8.508,8.453,7.700,0.753,0.65
4,38,0.2,1,3.250,3.232,2.852,0.380,0.55
5,38,0.2,2,3.745,3.714,2.991,0.724,0.82
6,38,0.4,1,6.148,6.128,5.834,0.294,0.33
7,38,0.4,2,7.670,7.614,6.979,0.635,0.73
8,40,0.2,1,2.314,2.299,2.066,0.233,0.64
9,40,0.2,2,2.885,2.855,2.356,0.499,1.03


In [10]:
# 1) Reference values from Longstaff & Schwartz (2001), Table 1 
REFERENCE = [
    # (S0, sigma, T, paper_value)
    (36, 0.20, 1, 4.478),
    (36, 0.20, 2, 4.840),
    (36, 0.40, 1, 7.101),
    (36, 0.40, 2, 8.508),
    (38, 0.20, 1, 3.250),
    (38, 0.20, 2, 3.745),
    (38, 0.40, 1, 6.148),
    (38, 0.40, 2, 7.670),
    (40, 0.20, 1, 2.314),
    (40, 0.20, 2, 2.885),
    (40, 0.40, 1, 5.312),
    (40, 0.40, 2, 6.920),
]

# 2) Common parameters
R = 0.06
K = 40
M_STEPS = 50       # 50 exercise points per year
N_PATHS = 50_000
SEED = 123

# 3) Define basis‐builder callables, now including Weighted Laguerre
BASES = [
    (
        "monomial",
        lambda S_itm: lsm._build_basis(S_itm, degree=4)
    ),
    (
        "monomial_cross",
        lambda S_itm: lsm._build_basis_with_cross(S_itm, degree=4)
    ),
    (
        "laguerre",
        lambda S_itm: lsm._build_basis_laguerre(S_itm, K=K, degree=4)
    ),
    (
        "laguerre_multid",
        lambda S_itm: lsm._build_basis_laguerre_multid(S_itm, K_vec=[K], p_max=4)
    ),
    (
        "weighted_laguerre",
        lambda S_itm: lsm._build_basis_weighted_laguerre(S_itm,K=K, degree=4)
    ),
]

# 4) Loop over basis functions and reference cases, with antithetic variates enabled
rows = []
for basis_label, basis_fn in BASES:
    for S0, sigma, T, paper_val in REFERENCE:
        price = lsm.lsm_price_multi(
            S0=[S0],
            r=R,
            sigma=[sigma],
            corr=[[1]],
            T=T,
            M=int(M_STEPS * T),
            N=N_PATHS,
            payoff_fn=lsm.payoff_put(K),
            basis_fn=basis_fn,
            seed=SEED,
            antithetic=True         # <— enable antithetic variates
        )
        abs_err = price - paper_val
        rel_err = abs(abs_err) / paper_val * 100
        rows.append({
            "Basis": basis_label,
            "S0": S0,
            "σ": sigma,
            "T": T,
            "Paper": paper_val,
            "Price": round(price, 3),
            "AbsErr": round(abs_err, 3),
            "RelErr (%)": round(rel_err, 2),
        })

df = pd.DataFrame(rows)

# 5) Pivot into a more readable table: one block per (S0, σ, T), columns for each basis
pivot = df.pivot_table(
    index=["S0", "σ", "T", "Paper"],
    columns="Basis",
    values="Price"
).reset_index().sort_values(["S0", "σ", "T"])

display(pivot)

# 6) Save full comparison (including error) to CSV
df.to_csv("lsm_basis_comparison.csv", index=False)
print("Saved detailed comparison to lsm_basis_comparison.csv")


Basis,S0,σ,T,Paper,laguerre,laguerre_multid,monomial,monomial_cross,weighted_laguerre
0,36,0.2,1,4.478,4.471,4.459,4.470,4.459,4.472
1,36,0.2,2,4.840,4.834,4.818,4.834,4.818,4.843
2,36,0.4,1,7.101,7.107,7.088,7.107,7.088,7.110
3,36,0.4,2,8.508,8.507,8.493,8.507,8.493,8.508
4,38,0.2,1,3.250,3.255,3.238,3.256,3.238,3.255
5,38,0.2,2,3.745,3.749,3.738,3.749,3.738,3.746
6,38,0.4,1,6.148,6.151,6.137,6.151,6.137,6.163
7,38,0.4,2,7.670,7.674,7.651,7.674,7.651,7.669
8,40,0.2,1,2.314,2.323,2.313,2.323,2.313,2.322
9,40,0.2,2,2.885,2.889,2.881,2.889,2.881,2.891


Saved detailed comparison to lsm_basis_comparison.csv
